<a href="https://colab.research.google.com/github/shahdabuhassanien/mini_DeepLearning-Projects-/blob/main/Regression_Models_in_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Deep Learning Tools
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout,Dense
from tensorflow.keras.metrics import MeanAbsoluteError,RootMeanSquaredError
# preprocessing Tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from sklearn import set_config
set_config(transform_output='pandas')

In [2]:
# Custom function for plotting each metric
def plot_history(history, figsize=(6,12), marker='o'):

    # Get list of metrics from history
    metrics = [c for c in history.history if not c.startswith('val_')]

    ## Separate row for each metric
    fig, axes = plt.subplots(nrows=len(metrics),figsize=figsize)

    # For each metric
    for i, metric_name in enumerate(metrics):

        # Get the axis for the current metric
        ax = axes[i]

        # Get metric from history.history
        metric_values = history.history[metric_name]
        # Get epochs from history
        epochs = history.epoch

        # Plot the training metric
        ax.plot(epochs, metric_values, label=metric_name, marker=marker)

        ## Check if val_{metric} exists. if so, plot:
        val_metric_name = f"val_{metric_name}"
        if val_metric_name in history.history:
            # Get validation values and plot
            metric_values = history.history[val_metric_name]
            ax.plot(epochs,metric_values,label=val_metric_name, marker=marker)

        # Final subplot adjustments
        ax.legend()
        ax.set_title(metric_name)
    fig.tight_layout()

    return fig, axes

In [3]:
fpath="https://docs.google.com/spreadsheets/d/e/2PACX-1vTg36jLawSOgGP9hp0oJ3OYZiHMWbuGLiau-8DMjtcKNv7v9Zy_zFBQs9gZU-44GGeIyfXE2iwo26_z/pub?output=csv"
df=pd.read_csv(fpath)
df=df.drop(columns="car name")
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8,307.0,130,3504,12.0,70,1
1,15.0,8,350.0,165,3693,11.5,70,1
2,18.0,8,318.0,150,3436,11.0,70,1
3,16.0,8,304.0,150,3433,12.0,70,1
4,17.0,8,302.0,140,3449,10.5,70,1


In [4]:
# check missing values and duplicates
print('missing values', df.info())
print('\nduplicated rows', df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    int64  
 4   weight        392 non-null    int64  
 5   acceleration  392 non-null    float64
 6   model year    392 non-null    int64  
 7   origin        392 non-null    int64  
dtypes: float64(3), int64(5)
memory usage: 24.6 KB
missing values None

duplicated rows 0


In [5]:
# now we are gonne split the data
y=df["mpg"]
x=df.drop(columns="mpg")
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)

In [6]:
cat_cols = ['cylinders','model year','origin']
num_cols = x_train.columns.drop(cat_cols)
# num preprocessing
scaler=StandardScaler()
num_tuple=("numeric",scaler,num_cols)

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_tuple=("Categorical",encoder,cat_cols)

preprocessor=ColumnTransformer([num_tuple,cat_tuple],verbose_feature_names_out=False)
x_train_tf=preprocessor.fit_transform(x_train)
x_test_tf=preprocessor.transform(x_test)


In [7]:
!pip install ydata-profiling

This Code bekow we Can use it to Have a quick EDA for Data

In [ ]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(df, title="EDA Report", explorative=True)
# profile.to_file("eda_report.html")

In [11]:
# from google.colab import files
# files.download("eda_report.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Build and Train the Model

In [9]:
# first we are gonna define the shape
input_shape=x_train_tf.shape[1]
input_shape

25

In [10]:
# we are gonna Creat a function to creaat
def build_model():
  model=Sequential()
  model.add(Dense(10,input_dim=input_shape,activation="relu"))
  model.add(Dense(10,input_dim=input_shape,activation="relu"))
  model.add(Dense(10,activation = 'relu'))
  model.add(Dropout(.2))
  model.add(Dense(1, activation = 'linear'))

  # here we are gonna compile the model
  model.compile (loss="mse",optimizer = 'adam',metrics=[MeanAbsoluteError(),RootMeanSquaredError()])
  return model